In [35]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Amazon
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv
import numpy as np
from sklearn.model_selection import GridSearchCV

dataset = 'Photo'
path = osp.join('data', dataset)
dataset = Amazon(path, dataset, transform=T.NormalizeFeatures())
dataset.shuffle()
data = dataset[0]
data.train_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.train_mask[:200] = True
data.val_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.val_mask[200:700] = True
data.test_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.test_mask[700:1700] = True

torch.manual_seed(0)

class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels, config):
        super(Net, self).__init__()
        self.conv1 = GATConv(in_channels, 8, heads=config[1], dropout=0.3)
        # On the Pubmed dataset, use heads=8 in conv2.
        self.conv2 = GATConv(config[1] * 8, out_channels, heads=1, concat=False,
                             dropout=0.3)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = (config[3])(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)



In [33]:
def train(data, config):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs

def loop(data, config):
    model = Net(dataset.num_features, dataset.num_classes, config).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0)
    
    print()
    print(f'Depth: {config[0]}, Heads: {config[1]}, Epochs: {config[2]}, Activation: {config[3]}')
    avg = np.empty([1,])
    max_val = 0
    early_stop_cnt = 0
    for epoch in range(1, 1001):
        train(data, config)
        train_acc, val_acc, test_acc = test(data)

        if val_acc > max_val:
            early_stop_cnt = 0
            max_val = val_acc
        else:
            early_stop_cnt += 1
            if (early_stop_cnt == config[2]):
                print(f'Early Stopping at epoch {epoch:03d}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')
                break

        avg = np.append(avg, test_acc)
        last_hun = np.average(avg[len(avg)-100:])
    
    #print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
    #      f'Test: {test_acc:.4f}, Avg100: {last_hun:.4f}')

In [37]:
from itertools import product

depth = [1]
heads = range(1,8)
epochs = range(50,300,50)
act = [F.elu, F.relu, F.tanh]
for config in list(product(depth, heads, epochs, act)):
    loop(data, config)
  


Depth: 1, Heads: 1, Epochs: 50, Activation: <function elu at 0x0000016CD9CADE50>
Early Stopping at epoch 112, Val: 0.8980, Test: 0.8940

Depth: 1, Heads: 1, Epochs: 50, Activation: <function relu at 0x0000016CD9CADC10>
Early Stopping at epoch 093, Val: 0.8960, Test: 0.8910

Depth: 1, Heads: 1, Epochs: 50, Activation: <function tanh at 0x0000016CD9CAE700>
Early Stopping at epoch 114, Val: 0.8960, Test: 0.9000

Depth: 1, Heads: 1, Epochs: 100, Activation: <function elu at 0x0000016CD9CADE50>
Early Stopping at epoch 108, Val: 0.8920, Test: 0.8850

Depth: 1, Heads: 1, Epochs: 100, Activation: <function relu at 0x0000016CD9CADC10>
Early Stopping at epoch 119, Val: 0.8960, Test: 0.8960

Depth: 1, Heads: 1, Epochs: 100, Activation: <function tanh at 0x0000016CD9CAE700>
Early Stopping at epoch 123, Val: 0.8900, Test: 0.8940

Depth: 1, Heads: 1, Epochs: 150, Activation: <function elu at 0x0000016CD9CADE50>
Early Stopping at epoch 325, Val: 0.8960, Test: 0.8930

Depth: 1, Heads: 1, Epochs: 150,

Early Stopping at epoch 338, Val: 0.8920, Test: 0.8970

Depth: 1, Heads: 5, Epochs: 50, Activation: <function elu at 0x0000016CD9CADE50>
Early Stopping at epoch 115, Val: 0.8980, Test: 0.8980

Depth: 1, Heads: 5, Epochs: 50, Activation: <function relu at 0x0000016CD9CADC10>
Early Stopping at epoch 060, Val: 0.8960, Test: 0.8950

Depth: 1, Heads: 5, Epochs: 50, Activation: <function tanh at 0x0000016CD9CAE700>
Early Stopping at epoch 118, Val: 0.8980, Test: 0.9000

Depth: 1, Heads: 5, Epochs: 100, Activation: <function elu at 0x0000016CD9CADE50>
Early Stopping at epoch 122, Val: 0.8960, Test: 0.8940

Depth: 1, Heads: 5, Epochs: 100, Activation: <function relu at 0x0000016CD9CADC10>
Early Stopping at epoch 133, Val: 0.8840, Test: 0.8840

Depth: 1, Heads: 5, Epochs: 100, Activation: <function tanh at 0x0000016CD9CAE700>
Early Stopping at epoch 200, Val: 0.8860, Test: 0.8860

Depth: 1, Heads: 5, Epochs: 150, Activation: <function elu at 0x0000016CD9CADE50>
Early Stopping at epoch 306, Val: